In [1]:

from libcst import parse_module, Module, Expr, Pass, Comment, CSTTransformer, Comparison, CSTNode, ComparisonTarget
from libcst import matchers
from typing import List, Dict, Union, Tuple, Type
import re
from libcst import Equal, GreaterThanEqual


In [2]:
class Mutation:
    def __init__(self, forward_transformer, reverse_transformer):
        self.forward_transformer = forward_transformer
        self.reverse_transformer = reverse_transformer

    def visit(self, tree):
        tree = tree.visit(self.forward_transformer)
        self.metadata = self.forward_transformer.metadata
        return tree


In [3]:

class IncorrectComparisonOperatorTransformer(CSTTransformer):
    def __init__(self, correct_operators, incorrect_operators):
        self.correct_operators = correct_operators
        self.incorrect_operators = incorrect_operators
        self.metadata = {}
        self.visit_functions = [self.visit_Comparison]


    def visit_Comparison(self, node: Comparison):
        for comparison in node.comparisons:
            for correct, incorrect in zip(self.correct_operators, self.incorrect_operators):
                if isinstance(comparison.operator, correct):
                    self.metadata[comparison] = (correct, incorrect)

    def leave_Comparison(self, original_node: Comparison, updated_node: Comparison) -> Comparison:
        for comparison, (correct, incorrect) in self.metadata.items():
            updated_comparison = comparison.with_changes(operator=incorrect())
            updated_node = original_node.with_changes(comparisons=[updated_comparison])
        return updated_node




In [4]:
class Mutator:
    def __init__(self, mutations:List[Mutation]):
        self.mutations = {mutation.__class__: mutation for mutation in mutations}

    def bug(self, script:str):
        self.module = parse_module(script)
        metadata = []
        for mutation in self.mutations.values():
            self.out_module = mutation.visit(self.module)
            metadata.append((mutation.__class__, mutation.metadata))
        return self.out_module.code, metadata

    def debug(self, script:str, metadata:List[Tuple[Type[Mutation], Dict]]):
        self.module = parse_module(script)
        for mutation_type, meta in metadata:
            mutation = self.mutations[mutation_type]
            self.module = mutation.reverse(meta)
        return self.module.code


In [5]:


# Original code
code = "a == b"
print("Original code:")
print(code)

# Create the transformer with the correct and incorrect operators
transformer = IncorrectComparisonOperatorTransformer([Equal], [GreaterThanEqual])

# Parse the code into a CST
module = parse_module(code)

# Apply the transformer to the CST
updated_module = module.visit(transformer)

# Print the updated code
print("Updated code using the transformer directly:")
print(updated_module.code) # Output: "a >= b"


Original code:
a == b
Updated code using the transformer directly:
a >= b


In [6]:
mutation = Mutation(transformer, None)
bugged_code, metadata = Mutator([mutation]).bug(code)
print("Bugged code:")
print(bugged_code)

Bugged code:
a >= b
